In [37]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import os
import pandas as pd

In [38]:
FB_EMAIL = 'ibrahim.mahmood7@hotmail.com'
FB_PASSWORD = 'sh3637Gdhj'

In [ ]:
def optimized_scroll_and_collect_urls(driver, max_wait_time=5):
    scroll_pause_time = 0.5  # Time to wait after each scroll (in seconds)
    last_height = driver.execute_script("return document.body.scrollHeight")
    start_time = time.time()
    
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        # If the height hasn't changed, we've reached the bottom
        if new_height == last_height:
            # Break the loop if we are done scrolling, but make sure enough time has passed to load the final content
            if time.time() - start_time > max_wait_time:
                break
        else:
            # Reset the start time if new content is loaded
            start_time = time.time()
        
        last_height = new_height
    
    # Once fully scrolled, get the page HTML
    page_html = driver.page_source
    print("Fully loaded page HTML has been retrieved.")
    urls = extract_urls_from_html(page_html)
    
    return urls

def extract_urls_from_html(members_html):
    soup = BeautifulSoup(members_html, 'html.parser')
    links = soup.find_all('a', href=True)
    urls = [link['href'] for link in links]

    return urls

def visit_public_group_and_get_all_member_urls(link):
    service = Service('chromedriver.exe')
    driver = webdriver.Chrome(service=service)
    
    try:
        login_to_facebook_and_redirect(driver, link)

        # Add a delay to ensure the group page loads completely
        time.sleep(3)

        click_members_link(driver)

        member_urls = optimized_scroll_and_collect_urls(driver)
        
        return member_urls

    finally:
        driver.quit()

def login_to_facebook_and_redirect(driver, group_url):
    driver.get('https://www.facebook.com/login')
    driver.find_element(By.ID, "email").send_keys(FB_EMAIL)
    driver.find_element(By.ID, "pass").send_keys(FB_PASSWORD)
    driver.find_element(By.NAME, "login").click()

    time.sleep(5)
    driver.get(group_url)
    print("Redirected to the Facebook group.")

def click_members_link(driver):
    try:
        # Wait for the members link to appear and be clickable
        members_link = WebDriverWait(driver, 4).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "a[href$='/members/']"))
        )

        driver.execute_script("arguments[0].scrollIntoView(true);", members_link)
        time.sleep(1)  # Give some time for scrolling
        driver.execute_script("arguments[0].click();", members_link)
        print("Clicked on the 'members' link.")

    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
file_path = r'fb_groups_data.xlsx'
df1 = pd.read_excel(file_path, skiprows=1, header=0)

In [45]:
df1.head()

,SNO,PAGES,FACEBOOK ID,NO OF MEMBERS,REMARKS
0,1,Cyber Security Group - Pakistan,https://www.facebook.com/groups/596000380410413/,Public Â· 9K members Â· 5 posts a month,NaN
1,2,Pakistani moms in Canada ðŸ‡¨ðŸ‡¦,https://www.facebook.com/groups/1915352918584025/,Public Â· 9K members Â· 10+ posts a day,NaN
2,3,Electrical engineering world,https://www.facebook.com/groups/303168311067353/,Public Â· 96K members Â· 10+ posts a day,NaN
3,4,Gold Rate Pakistan Daily Update,https://www.facebook.com/groups/1236463527108778/,Public Â· 94K members Â· 3 posts a day,NaN
4,5,The Bankers Forum - (Pakistan),https://www.facebook.com/groups/TheBankersForum/,Public Â· 93K members Â· 6 posts a day,NaN


In [46]:
group_url = df1['FACEBOOK ID'].iloc[52]

In [47]:
group_url

'https://www.facebook.com/groups/807147959841449/'

In [48]:
all_member_urls = visit_public_group_and_get_all_member_urls(group_url)

Redirected to the Facebook group.
Clicked on the 'members' link.
Fully loaded page HTML has been retrieved.


In [27]:
all_member_urls

['/',
 '/',
 '/friends/',
 '/marketplace/?ref=app_tab',
 '/gaming/play/?store_visit_source=gaming_tab',
 '/notifications/',
 'https://www.facebook.com/photo/?fbid=275793300634453&set=p.275793300634453',
 'https://www.facebook.com/groups/807147959841449/',
 '/groups/807147959841449/members/',
 '/groups/807147959841449/about/',
 'https://www.facebook.com/groups/807147959841449/',
 '/groups/807147959841449/members/',
 '/groups/807147959841449/events/',
 '/groups/807147959841449/media/',
 '/groups/807147959841449/files/files/',
 'https://www.facebook.com/groups/807147959841449/',
 'https://www.facebook.com/help/727473118066542',
 '/groups/807147959841449/user/100046113546388/',
 '/groups/807147959841449/user/100046113546388/',
 '/groups/807147959841449/user/100055754507999/',
 '/groups/807147959841449/user/100055754507999/',
 '/groups/807147959841449/user/100054619163582/',
 '/groups/807147959841449/user/100054619163582/',
 '/groups/807147959841449/members/things_in_common/',
 '/groups/807

In [29]:
x = extract_member_urls(all_member_urls)
x

['/groups/807147959841449/user/100046113546388/',
 '/groups/807147959841449/user/100046113546388/',
 '/groups/807147959841449/user/100055754507999/',
 '/groups/807147959841449/user/100055754507999/',
 '/groups/807147959841449/user/100054619163582/',
 '/groups/807147959841449/user/100054619163582/',
 '/groups/807147959841449/user/100055754507999/',
 '/groups/807147959841449/user/100055754507999/',
 '/groups/807147959841449/user/100054619163582/',
 '/groups/807147959841449/user/100054619163582/',
 '/groups/807147959841449/user/100014492740176/',
 '/groups/807147959841449/user/100014492740176/',
 '/groups/807147959841449/user/100054619163582/',
 '/groups/807147959841449/user/100054619163582/',
 '/groups/807147959841449/user/100083674766433/',
 '/groups/807147959841449/user/100083674766433/',
 '/groups/807147959841449/user/100042912082452/',
 '/groups/807147959841449/user/100042912082452/',
 '/groups/807147959841449/user/100027678798896/',
 '/groups/807147959841449/user/100027678798896/',


In [33]:
a = []
for i in x:
    a.append(convert_to_proper_url(i))

In [34]:
a

['https://www.facebook.com/profile.php?id=100046113546388',
 'https://www.facebook.com/profile.php?id=100046113546388',
 'https://www.facebook.com/profile.php?id=100055754507999',
 'https://www.facebook.com/profile.php?id=100055754507999',
 'https://www.facebook.com/profile.php?id=100054619163582',
 'https://www.facebook.com/profile.php?id=100054619163582',
 'https://www.facebook.com/profile.php?id=100055754507999',
 'https://www.facebook.com/profile.php?id=100055754507999',
 'https://www.facebook.com/profile.php?id=100054619163582',
 'https://www.facebook.com/profile.php?id=100054619163582',
 'https://www.facebook.com/profile.php?id=100014492740176',
 'https://www.facebook.com/profile.php?id=100014492740176',
 'https://www.facebook.com/profile.php?id=100054619163582',
 'https://www.facebook.com/profile.php?id=100054619163582',
 'https://www.facebook.com/profile.php?id=100083674766433',
 'https://www.facebook.com/profile.php?id=100083674766433',
 'https://www.facebook.com/profile.php?i

---

In [28]:
def extract_member_urls(urls):
    # Define the base path that identifies member URLs
    member_url_pattern = "/user/"
    
    # Filter the URLs that contain the member URL pattern
    member_urls = [url for url in urls if member_url_pattern in url]
    
    return member_urls

In [30]:
def convert_to_proper_url(broken_url):
    numeric_id = broken_url.split('/')[-2]  
    proper_url = f"https://www.facebook.com/profile.php?id={numeric_id}"
    
    return proper_url

In [31]:
def func2(urls):
    list1 = []
    urls2 = extract_member_urls(urls)
    
    for url in urls2:
        list1.append(convert_to_proper_url(url))
    
    final_set_of_urls = set(list1)

    return list(final_set_of_urls)

---

In [ ]:
def login_to_facebook(driver):
    driver.get('https://www.facebook.com/login')
    driver.find_element(By.ID, "email").send_keys(FB_EMAIL)
    driver.find_element(By.ID, "pass").send_keys(FB_PASSWORD)
    driver.find_element(By.NAME, "login").click()
    time.sleep(5)  # Wait for login to complete
    print("Logged in to Facebook.")

def optimized_scroll_and_collect_urls(driver, max_wait_time=10):
    scroll_pause_time = 0.5  # Time to wait after each scroll (in seconds)
    last_height = driver.execute_script("return document.body.scrollHeight")
    start_time = time.time()
    
    while True:
        # Scroll down to the bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        # Check if more content is loaded
        if new_height == last_height:
            # Break the loop if no new content has loaded for a while
            if time.time() - start_time > max_wait_time:
                break
        else:
            # Reset the start time if new content is loaded
            start_time = time.time()
        
        last_height = new_height
    
    # Once fully scrolled, get the page HTML
    page_html = driver.page_source
    urls = extract_urls_from_html(page_html)
    
    print(f"Collected {len(urls)} URLs from the People section.")
    return urls

def extract_urls_from_html(members_html):
    soup = BeautifulSoup(members_html, 'html.parser')
    links = soup.find_all('a', href=True)
    urls = [link['href'] for link in links]
    return urls
def click_members_link(driver):
    try:
        # Wait for the tab buttons to appear and collect all the potential links
        tab_links = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[role='tab']"))
        )

        for tab_link in tab_links:
            # Get the text and href attributes safely
            tab_text = tab_link.text.strip() if tab_link.text else ""
            tab_href = tab_link.get_attribute('href') if tab_link.get_attribute('href') else ""

            # Convert to lowercase for comparison
            tab_text = tab_text.lower()
            tab_href = tab_href.lower()
            
            # Check if the tab matches the "People" or "Members" link criteria
            if 'people' in tab_text or 'members' in tab_text or '/members/' in tab_href or '/people/' in tab_href:
                driver.execute_script("arguments[0].scrollIntoView(true);", tab_link)
                time.sleep(1)  # Give some time for scrolling
                driver.execute_script("arguments[0].click();", tab_link)
                print(f"Clicked on the '{tab_text.capitalize()}' link.")
                return True
        
        print("Members/People section not found.")
        return False

    except Exception as e:
        print(f"An error occurred while trying to click the 'People' or 'Members' link: {e}")
        return False



def visit_public_group_and_get_all_member_urls(driver, group_url):
    try:
        driver.get(group_url)
        time.sleep(3)  # Ensure the group page is fully loaded

        # Click the "People" or "Members" link
        if click_members_link(driver):
            # Collect all URLs after fully scrolling the page
            member_urls = optimized_scroll_and_collect_urls(driver)
            return member_urls
        else:
            print("Failed to navigate to the 'People' section.")
            return []

    except Exception as e:
        print(f"Error visiting group: {e}")
        return []

def extract_member_urls(urls):
    member_url_pattern = "/user/"
    member_urls = [url for url in urls if member_url_pattern in url]
    return member_urls

def convert_to_proper_url(broken_url):
    numeric_id = broken_url.split('/')[-2]  
    proper_url = f"https://www.facebook.com/profile.php?id={numeric_id}"
    return proper_url

def process_member_urls(urls):
    list1 = []
    urls2 = extract_member_urls(urls)
    
    for url in urls2:
        list1.append(convert_to_proper_url(url))
    
    final_set_of_urls = set(list1)
    return list(final_set_of_urls)

def retrieve_and_process_member_urls(df1, index=52):
    group_url = df1.loc[index, 'FACEBOOK ID']
    group_name = df1.loc[index, 'PAGES']
    
    # Set up the WebDriver
    service = Service(r'chromedriver.exe')
    driver = webdriver.Chrome(service=service)
    
    try:
        # Step 1: Log in to Facebook
        login_to_facebook(driver)
        
        # Step 2: Visit the group and get all member URLs
        all_member_urls = visit_public_group_and_get_all_member_urls(driver, group_url)
        
        # Step 3: Process the URLs to get the final list of profile URLs
        final_urls = process_member_urls(all_member_urls)
        
        # Step 4: Create a DataFrame with the required columns
        result_df = pd.DataFrame({
            'Group Name': [group_name] * len(final_urls),
            'Member URL': final_urls
        })
        
        return result_df
    
    finally:
        driver.quit()

# Assuming df1 is already defined and contains the necessary data
result_df = retrieve_and_process_member_urls(df1, index=52)


Logged in to Facebook.
Clicked on the 'People' link.
Collected 51 URLs from the People section.


In [64]:
df1.iloc[52]

SNO                                                            53
PAGES                    PAKISTAN PROPERTY COUNCIL (0332-2638077)
FACEBOOK ID      https://www.facebook.com/groups/807147959841449/
NO OF MEMBERS             Public Â· 81 members Â· 10 posts a year
REMARKS                                                       NaN
Name: 52, dtype: object

In [63]:
result_df.iloc[1]

Group Name             PAKISTAN PROPERTY COUNCIL (0332-2638077)
Member URL    https://www.facebook.com/profile.php?id=100007...
Name: 1, dtype: object

In [60]:
result_df.shape

(12, 2)

In [ ]:
# Check for duplicates in the 'Member URL' column
duplicates = result_df['Member URL'].duplicated()

# Display duplicate entries
duplicate_entries = result_df[duplicates]

# Print the number of duplicate entries and the duplicate URLs
print(f"Number of duplicate member URLs: {duplicates.sum()}")
print(duplicate_entries)